______________________________________________________________________________________________________
This notebook contains a simple variational autoencoder. 

The following resources have been helpful:
* https://arxiv.org/pdf/1606.05908v2.pdf
* https://jmetzen.github.io/2015-11-27/vae.html
_______________________________________________________________________________________________________

# Setup

In [1]:
from __future__ import print_function

import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
sys.path.append('..')
from models.VAE import VAE
from models.CVAE import CVAE

np.random.seed(0)
tf.set_random_seed(0)

In [2]:
# load data
import models.DataReader as Data

# get data handler
data_type = 'mnist'
data_dir = '/home/mattw/Dropbox/git/dreamscape/data/'

if data_type is 'mnist':
    data = Data.DataReaderMNIST(data_dir, one_hot=False)
elif data_type is 'cifar':
    data = Data.DataReaderCIFAR(data_dir, one_hot=False)
elif data_type is 'imagenet':
    data = Data.DataReaderImagenet(data_dir, one_hot=False)      

Extracting /home/mattw/Dropbox/git/dreamscape/data/train-images-idx3-ubyte.gz
Extracting /home/mattw/Dropbox/git/dreamscape/data/train-labels-idx1-ubyte.gz
Extracting /home/mattw/Dropbox/git/dreamscape/data/t10k-images-idx3-ubyte.gz
Extracting /home/mattw/Dropbox/git/dreamscape/data/t10k-labels-idx1-ubyte.gz


# Define and Train a VAE

In [3]:
save_file = 0
save_dir = '/media/data/Dropbox/Git/dreamscape/tmp/vae-mnist.ckpt'
net_type = 'vae' # 'vae' | 'cvae'

# define model params
layers_encoder = [784, 400, 400]
layer_latent = 20
layers_decoder = [400, 400, 784]
num_categories = 10

# define training params
batch_size = 100
training_epochs = 10
display_epochs = 1
use_gpu = 1

# initialize network
if net_type is 'vae':
    net = VAE(layers_encoder=layers_encoder, 
              layer_latent=layer_latent,
              layers_decoder=layers_decoder,
              learning_rate=1e-3)
elif net_type is 'cvae':
    net = CVAE(layers_encoder=layers_encoder, 
              layer_latent=layer_latent,
              layers_decoder=layers_decoder,
              num_categories=num_categories,
              learning_rate=1e-3)

# start the tensorflow session
config = tf.ConfigProto(device_count = {'GPU': use_gpu})
sess = tf.Session(config=config, graph=net.graph)
sess.run(net.init)

# train network
time_start = time.time()
net.train(sess, 
          data=data,
          batch_size=batch_size,
          training_epochs=training_epochs,
          display_epochs=display_epochs)
time_end = time.time()
print('time_elapsed: %g' % (time_end - time_start))

# save network
if save_file:
    net.save_model(sess, save_dir)

# close the tensorflow session
# sess.close()

Epoch 000: cost = 25.80391
Epoch 001: cost = 23.39750
Epoch 002: cost = 23.63233
Epoch 003: cost = 22.41494
Epoch 004: cost = 21.11941
Epoch 005: cost = 21.37671
Epoch 006: cost = 20.60849
Epoch 007: cost = 20.86390
Epoch 008: cost = 21.95274
Epoch 009: cost = 20.99997
time_elapsed: 35.5386


## Reload already trained model

In [ ]:
# define model params
layers_encoder = [784, 400, 400]
layer_latent = 20
layers_decoder = [400, 400, 784]
num_categories = 10

# define training params
batch_size = 100
training_epochs = 40
display_epochs = 1
use_gpu = 1

# initialize network
if net_type is 'vae':
    net = VAE(layers_encoder=layers_encoder, 
              layer_latent=layer_latent,
              layers_decoder=layers_decoder,
              learning_rate=1e-3,
              data_dir='/home/mattw/Dropbox/git/dreamscape/data/',
              data_type='mnist')
elif net_type is 'cvae':
    net = CVAE(layers_encoder=layers_encoder, 
              layer_latent=layer_latent,
              layers_decoder=layers_decoder,
              num_categories=num_categories,
              learning_rate=1e-3,
              data_dir='/home/mattw/Dropbox/git/dreamscape/data/',
              data_type='mnist')

# start the tensorflow session
config = tf.ConfigProto(device_count = {'GPU': use_gpu})
sess = tf.Session(config=config, graph=net.graph)
sess.run(net.init)

# restore previously trained model
net.load_model(sess, '/media/data/Dropbox/Git/dreamscape/tmp/vae-mnist.ckpt')

# Visualize Model

## Reconstruction Visualization

In [4]:
x = data.train.next_batch(net.batch_size)
eps = np.zeros((net.batch_size, net.num_lvs))
recon = net.reconstruct(sess, x[0], eps)

f, ax = plt.subplots(2,5)
for j in range(5):
    ax[0,j].imshow(np.reshape(x[0][j,:], (28, 28)),
                  interpolation="nearest",
                  cmap="gray")
    ax[0,j].axes.get_xaxis().set_visible(False)
    ax[0,j].axes.get_yaxis().set_visible(False)
    
    ax[1,j].imshow(np.reshape(recon[j,:], (28, 28)),
                  interpolation="nearest",
                  cmap="gray")
    ax[1,j].axes.get_xaxis().set_visible(False)
    ax[1,j].axes.get_yaxis().set_visible(False)

plt.show()

## Latent Space Visualization I

In [ ]:
nx = ny = 20
x_values = np.linspace(-3, 3, nx)
y_values = np.linspace(-3, 3, ny)

canvas = np.empty((28*ny, 28*nx))
for i, yi in enumerate(x_values):
    for j, xi in enumerate(y_values):
        z_mean = np.array([[xi, yi]])
        x_mean = net.generate(sess, z_mean=z_mean)
        canvas[(nx-i-1)*28:(nx-i)*28, j*28:(j+1)*28] = x_mean[0].reshape(28, 28)

plt.figure(figsize=(8, 10))        
Xi, Yi = np.meshgrid(x_values, y_values)
plt.imshow(canvas, origin="upper",
           interpolation="nearest",
           cmap="gray")
plt.tight_layout()
plt.show()

## GIF Animation of Latent Space During Training

In [ ]:
import os
from io import BytesIO
import PIL.Image
import scipy.misc
import scipy.io
from IPython.display import clear_output, Image, display

In [ ]:
# define model params
layers_encoder = [784, 400, 400]
layer_latent = 2
layers_decoder = [400, 400, 784]

# initialize model
net = VAE(layers_encoder=layers_encoder, 
          layer_latent=layer_latent,
          layers_decoder=layers_decoder)

In [ ]:
# training details
batch_size = 100
display_iters = None
iters_per_image = np.logspace(0, 3, num=50).astype(int)

# image details
nx = ny = 20
x_values = np.linspace(-3, 3, nx)
y_values = np.linspace(-3, 3, ny)

# store processed images
dir_path = '/home/mattw/Desktop/test_movie'  
saving = 1

for epoch, iters in enumerate(iters_per_image):
    
    # output updates
    print('\rEpoch %03g of %03g - training for %05g iters' % 
          (epoch+1, iters_per_image.size, iters), end='')
    
    # train model
    net.train_iters(sess, batch_size=batch_size,
                          training_iters=iters,
                          display_iters=display_iters)
    
    # create latent state representation
    canvas = np.empty((28*ny, 28*nx))
    for i, yi in enumerate(x_values):
        for j, xi in enumerate(y_values):
            z_mean = np.array([[xi, yi]])
            x_mean = net.generate(sess, z_mean=z_mean)
            canvas[(nx-i-1)*28:(nx-i)*28, j*28:(j+1)*28] = x_mean[0].reshape(28, 28)
    
    # save output
    if saving:
        file_name = 'vae-mnist_epoch_%03i.jpg' % epoch
        file_path = '/'.join([dir_path, file_name])
        canvas = np.uint8(255*canvas)
        PIL.Image.fromarray(canvas).save(file_path, 'jpeg')


### to create a gif:
convert -delay 4x120 -loop 0 *.jpg animated.gif

take all jpgs in current directory and turn them into a gif that loops indefinitely, with a framerate of 120/4 = 30 fps

In [ ]:
%reload_ext watermark
%watermark -a "Matt Whiteway" -d -v -m -p numpy,tensorflow